# Module 1

## Essential File Modes: Read ('r'), Write ('w'), Append ('a')

1. 'r' (Read Mode): This is the default mode if no mode is explicitly specified. When a file is opened in read mode, the file pointer—an internal marker indicating the current position for reading—is placed at the very beginning of the file. If the specified file does not exist, attempting to open it in this mode will result in a FileNotFoundError. It is important to note that this mode only permits reading; writing operations are not allowed.
2. 'w' (Write Mode): This mode opens a file exclusively for writing. A critical characteristic of 'w' mode is its behavior when the file already exists: its entire contents are truncated (erased), effectively leaving an empty file. If the file does not exist, a new, empty file is created. The file pointer is positioned at the beginning of the file, ready for new content to be written.
3. 'a' (Append Mode): Append mode also opens a file for writing. Unlike 'w' mode, if the file already exists, its existing content is preserved. New data is written to the very end of the file, extending its content. If the file does not exist, a new, empty file is created. The file pointer is initially placed at the end of the file, ready for new additions.



In [1]:
# Example: Reading from a file (assuming 'my_file.txt exists)
# Create a dummy file for demostration 

In [2]:
with open('my_file.txt', 'w') as f:
    f.write('This is the first line.\n')
    f.write('This is the second line.')

file = open('my_file.txt', 'r')
content = file.read()
print("Content read in 'r' mode:")
print(content)
file.close()

Content read in 'r' mode:
This is the first line.
This is the second line.


In [3]:
# Example: Writing to a file (will overwrite if 'my_file.txt' exists)
file = open('my_file.txt', 'w')
file.write('Hello, Python File Handling!')
file.close()
print("\nContent after writing in 'w' mode:")
with open('my_file.txt', 'r') as f:
   print(f.read())


Content after writing in 'w' mode:
Hello, Python File Handling!


In [4]:
# Example: Appending to a file
file = open('my_file.txt', 'a')
file.write('\nThis line is appended.')
file.close()
print("\nContent after appending in 'a' mode:")
with open('my_file.txt', 'r') as f:
   print(f.read())


Content after appending in 'a' mode:
Hello, Python File Handling!
This line is appended.


## Combined Modes ('r+', 'w+', 'a+')

The '+' modifier can be combined with the primary modes ('r', 'w', or 'a') to enable both reading and writing operations on the file.

1. 'r+' (Read and Write): This mode opens the file for both reading and writing. The file pointer is initially at the beginning. If the file does not exist, a FileNotFoundError is raised. Crucially, existing content is not truncated, allowing for modifications within the file.
2. 'w+' (Write and Read): This mode opens the file for both writing and reading. If the file exists, its contents are truncated (erased), just like in 'w' mode. If the file does not exist, a new file is created. The file pointer is at the beginning.
3. 'a+' (Append and Read): This mode opens the file for both appending and reading. The file pointer is initially at the end of the file for writing new data. However, the pointer can be explicitly moved using seek() (discussed in Module 5) to read from other positions within the file. If the file does not exist, a new file is created.

A significant point of caution arises when using 'w' and 'w+' modes. These modes inherently carry a risk of silent data loss. Accidentally opening an existing and important file in either 'w' or 'w+' mode will immediately wipe out its entire contents without any warning or confirmation from Python. This behavior makes it imperative to implement robust error handling and to proactively check for file existence before performing any write operations, a topic that will be covered in detail in Module 4 and Module 8. Developers must be acutely aware of this truncation behavior to prevent unintended data destruction.


## Binary Mode ('b')
The 'b' modifier is used to open files in binary mode. This mode is specifically for handling non-human-readable files, such as images, audio files, compiled programs, or encrypted data. When a file is opened in binary mode, data is read and written as bytes objects rather than standard strings. This mode is always combined with other primary modes, forming combinations like 'rb' (read binary) or 'wb' (write binary).

In [5]:
# Example: Writing and reading binary data
# A simple byte sequence representing some arbitrary binary data
binary_data_to_write = b'\x00\xFF\x00\xFF\x1A\x0A'
with open('example.bin', 'wb') as f:
   f.write(binary_data_to_write)
print(f"\nBinary data written to 'example.bin': {binary_data_to_write}")

with open('example.bin', 'rb') as f:
   data = f.read()
   print(f"Binary data read from 'example.bin': {data}")


Binary data written to 'example.bin': b'\x00\xff\x00\xff\x1a\n'
Binary data read from 'example.bin': b'\x00\xff\x00\xff\x1a\n'


## The Critical Step: Closing Files with file.close()
After all operations on a file are completed, it is absolutely essential to close it using the file.close() method. This action is not merely a formality; it is a critical step that releases the system resources that the operating system allocated for your program to access that file. These resources are often referred to as "file handles" on Windows or "file descriptors" on Unix-like systems (including Linux and macOS).

Failing to close files can lead to several detrimental consequences:

    Data Corruption: Unsaved changes that are still buffered in memory might not be written to the physical disk, leading to incomplete or corrupted data.
    Resource Leaks: Your program might continue to hold onto system resources unnecessarily. Each open file consumes a file descriptor, and operating systems have limits on how many descriptors a single process can hold. Exhausting these limits can lead to performance degradation or prevent your program, or even other applications, from opening new files.
    File Locking: On some operating systems, an open file might remain locked, preventing other processes or even subsequent attempts by your own program from accessing or modifying it.

The underlying mechanism for file handling involves Python making system calls to the operating system. When open() is invoked, the OS locates the file and returns an integer identifier (the file handle/descriptor). This identifier is then used for all subsequent read, write, or close operations. Therefore, closing a file is not just a Python-specific instruction; it is a direct command to the operating system to release a valuable, limited resource. This concept extends beyond files to other system resources like network sockets or database connections, underscoring the broader importance of resource management, which is elegantly addressed by context managers (Module 3).

# Module 2: Reading and Writing Text Files

Once a file is successfully opened, the next step involves interacting with its content. This module explores the various methods Python provides for reading data from text files and writing new content to them.

## Reading Strategies

Python offers several methods to retrieve data from an opened file object, each suited for different scenarios and file sizes.

### read(): Grabbing the Whole Content

The read() method is designed to read the entire content of a file and return it as a single string. This is convenient for smaller files where loading the entire content into memory is not an issue. An optional integer argument, size, can be passed to read() to specify the number of characters (or bytes in binary mode) to read from the current file pointer position. If size is omitted or provided as a negative value, the method reads the entire remaining content of the file from the current pointer position until the end.

In [6]:
# Create a sample file for demonstration
with open('sample.txt', 'w') as f:
   f.write('Line 1: Hello World!\n')
   f.write('Line 2: Python is fun.\n')
   f.write('Line 3: File handling basics.')

# Read the entire file
with open('sample.txt', 'r') as f:
   content = f.read()
   print("--- Full Content ---")
   print(content)

# Read a specific number of characters
with open('sample.txt', 'r') as f:
   partial_content = f.read(10) # Reads first 10 characters
   print("\n--- Partial Content (10 chars) ---")
   print(partial_content)

--- Full Content ---
Line 1: Hello World!
Line 2: Python is fun.
Line 3: File handling basics.

--- Partial Content (10 chars) ---
Line 1: He


### readline(): Processing Line by Line (Efficient for Large Files)

The readline() method reads a single line from the file at a time, including the newline character (\n) at the end of the line. When the end of the file is reached, readline() returns an empty string (''). This method is particularly valuable for processing very large files because it avoids loading the entire file into memory simultaneously, thereby conserving system resources.

In [9]:
# Read line by line
with open('sample.txt', 'r') as f:
   print("\n--- Reading Line by Line ---")
   line1 = f.readline()
   print(line1, end='') # Use end='' to avoid double newline
   line2 = f.readline()
   print(line2, end='')
   # Demonstrate looping through lines
   print("\n--- Looping Line by Line ---")
   f.seek(0) # Reset pointer to beginning to re-read
   line = f.readline()
   while line:
       print(line, end='')
       line = f.readline()


--- Reading Line by Line ---
Line 1: Hello World!
Line 2: Python is fun.

--- Looping Line by Line ---
Line 1: Hello World!
Line 2: Python is fun.
Line 3: File handling basics.

### readlines(): Getting All Lines as a List

The readlines() method reads all remaining lines from the file from the current file pointer position until the end of the file. It then returns these lines as a list of strings, where each string in the list represents a line from the file and includes its respective newline character.

In [1]:
# Read all lines into a list
with open('sample.txt', 'r') as f:
   lines = f.readlines()
   print("\n--- All Lines as List ---")
   for line in lines:
       print(line, end='')


--- All Lines as List ---
Line 1: Hello World!
Line 2: Python is fun.
Line 3: File handling basics.

When choosing a reading strategy, it is important to consider memory efficiency and performance. While read() and readlines() offer convenience by loading entire file contents into RAM, this approach can quickly become problematic for files that are gigabytes in size, potentially leading to a MemoryError or significantly slowing down program execution. For processing large text files, the most memory-efficient method involves iterating directly over the file object itself (e.g., for line in file_object:). This implicit iteration leverages readline() internally, processing data one line at a time without holding the entire file in memory. This distinction is crucial for developing scalable applications capable of handling substantial datasets.

## Writing Data

Python provides methods to write string data to an opened file object.

### write(): Adding Text to Files

The write() method is used to write a string to the file at the current position of the file pointer. It returns the number of characters (or bytes in binary mode) that were successfully written. A key characteristic of write() is that it does not automatically append a newline character (\n) at the end of the string. If a line break is desired, it must be explicitly included within the string being written.

In [8]:
# Writing to a file (overwrites existing content if in 'w' mode)
with open('output.txt', 'w') as f:
   f.write('First line of text.\n')
   f.write('Second line of text.')
print("\n--- Content of output.txt after writing ---")
with open('output.txt', 'r') as f:
   print(f.read())


--- Content of output.txt after writing ---
First line of text.
Second line of text.


### writelines(): Writing Multiple Lines

The writelines() method is designed to write a sequence of strings (such as a list or tuple of strings) to the file. It is crucial to understand that writelines() does not automatically add line endings (\n) to each item in the sequence. The responsibility for including the appropriate newline characters within each string in the sequence falls entirely on the developer if they wish for the content to appear on separate lines in the file.

In [9]:
# Writing multiple lines from a list
lines_to_write = ['\nThird line of text.\n','Fourth line of text.'] 
with open('output.txt', 'a') as f: # Using 'a' to append to existing content
   f.writelines(lines_to_write)
print("\n--- Content of output.txt after appending with writelines ---")
with open('output.txt', 'r') as f:
   print(f.read())


--- Content of output.txt after appending with writelines ---
First line of text.
Second line of text.
Third line of text.
Fourth line of text.


A common pitfall for beginners using write() and writelines() is the expectation that these methods behave like the print() function, which automatically adds a newline character by default. However, this is not the case for file writing methods. Forgetting to explicitly include \n characters within the strings being written will result in all text being concatenated into a single, continuous, and often unreadable line within the output file. This detail significantly impacts the readability and subsequent parsability of the generated files, potentially disrupting downstream processing or human review.

# Module 3: The with Statement and Context Managers

This module introduces the most Pythonic and highly recommended approach to file handling: the with statement. It is a cornerstone of writing robust and reliable Python code, particularly when dealing with external resources.


## The Pythonic Way: Why with open(...) is Superior

While manually calling open() to acquire a file object and close() to release it is functional, this approach is inherently susceptible to errors. If an exception occurs during file operations—between the open() call and the close() call—the close() method might never be executed, leading to resource leaks and potential data corruption. The with statement, formally introduced in Python Enhancement Proposal (PEP) 343, provides a cleaner, safer, and more readable alternative for managing resources like files. It automatically handles both the setup (acquiring the resource, such as opening a file) and teardown (releasing the resource, such as closing the file) phases, guaranteeing that cleanup occurs even if the code encounters an error.

## Automatic Resource Management: No More Forgotten close() Calls

The primary and most significant advantage of using the with open(...) as file: construct is its guarantee that the file will be closed automatically once the indented block of code is exited. This closure occurs regardless of whether the block finishes normally or terminates prematurely due to an exception. This automatic resource management prevents common issues such as resource leaks (where file handles remain open indefinitely), ensures data integrity by flushing buffered changes to disk, and significantly simplifies the codebase by eliminating the need for explicit try...finally blocks dedicated solely to cleanup.

In [10]:
# Example: Using the with statement for writing
with open('example_with.txt', 'w') as file:
   file.write('This text is written using the with statement.')
   # No need to call file.close() explicitly!
print("File 'example_with.txt' created and automatically closed.")

# Example with error handling (still closes automatically)
try:
   with open('non_existent_file.txt', 'r') as file: # This file does not exist
       content = file.read()
       print(content)
except FileNotFoundError:
   print("Caught an error, but the file (if it existed) would be closed.")

File 'example_with.txt' created and automatically closed.
Caught an error, but the file (if it existed) would be closed.


## A Glimpse Under the Hood: __enter__ and __exit__ (Simplified)

The with statement's power stems from its interaction with "context manager" objects. Any object that is designed to be used with a with statement must implement two special methods, often referred to as "dunder" (double underscore) methods:

    __enter__(self): This method is invoked when the execution flow first enters the with block. Its primary role is to perform any necessary setup or resource acquisition (e.g., opening a file, establishing a database connection). It typically returns the resource object itself (e.g., the opened file object), which is then bound to the variable specified after the as keyword in the with statement.
    __exit__(self, exc_type, exc_value, exc_tb): This method is called when the execution flow leaves the with block, irrespective of whether the block completed successfully or terminated due to an exception. Its purpose is to perform any necessary cleanup or resource release (e.g., closing the file, committing database changes). The exc_type, exc_value, and exc_tb arguments provide detailed information about any exception that might have occurred within the with block. If __exit__ returns a True value, it indicates that the context manager has successfully handled and suppressed the exception, allowing execution to continue as if no error occurred. Conversely, if it returns False (or implicitly None, which is the default), the exception is re-raised and propagated outside the context manager.

The with statement, and the underlying concept of context managers, represents a powerful design pattern in Python for managing any resource that requires a defined setup and teardown procedure. This extends far beyond simple file handling to encompass a wide array of scenarios, such as managing network connections, database sessions, or even acquiring and releasing locks in concurrent programming. By abstracting away the boilerplate code for resource acquisition and release, context managers make code significantly cleaner, more readable, and inherently more robust. Understanding this pattern is crucial for interacting with many other Python libraries that leverage context managers, such as sqlite3.connect for database access or threading.Lock for managing shared resources in multi-threaded applications.

A subtle yet powerful aspect of the __exit__ method lies in its ability to suppress exceptions. While with statements are generally understood to automatically close resources, the detailed mechanism reveals that if __exit__() returns True, any exception that occurred within the with block is effectively "swallowed" or suppressed. This means a context manager can internally handle an error and allow the program to proceed as if no error had occurred. Although this capability offers immense power and flexibility in designing custom resource management, it must be approached with extreme caution. A context manager that silently suppresses critical errors without proper logging or alternative handling can make debugging incredibly challenging, as the root cause of an issue might be obscured. This underscores the importance of thoughtful context manager design and transparent error reporting in professional software development.

## Refactoring Examples with with

To illustrate the benefits, consider refactoring the basic file reading and writing examples from Module 2 using the with statement. The reduction in boilerplate code and the inherent safety improvements become immediately apparent.

In [11]:
# Original example (Module 2, read())
# file = open('sample.txt', 'r')
# content = file.read()
# print(content)
# file.close()

# Refactored with 'with' statement
print("\n--- Refactored Read Example with 'with' ---")
with open('sample.txt', 'r') as file:
   content = file.read()
   print(content)
# File is automatically closed here

# Original example (Module 2, write())
# file = open('output.txt', 'w')
# file.write('First line of text.\n')
# file.write('Second line of text.')
# file.close()

# Refactored with 'with' statement
print("\n--- Refactored Write Example with 'with' ---")
with open('output_refactored.txt', 'w') as file:
   file.write('This is the first line from refactored write.\n')
   file.write('This is the second line from refactored write.')
# File is automatically closed here
print("File 'output_refactored.txt' created and automatically closed.")


--- Refactored Read Example with 'with' ---
Line 1: Hello World!
Line 2: Python is fun.
Line 3: File handling basics.

--- Refactored Write Example with 'with' ---
File 'output_refactored.txt' created and automatically closed.


# Module 4: Robust File Handling: Error Management

Despite the advantages of the with statement, file operations are inherently susceptible to failures caused by external factors beyond a program's direct control. Files might be missing, access permissions could be denied, or the storage device might be full. Consequently, implementing robust error handling is not merely a good practice but an essential requirement for building reliable and resilient applications. Python signals these issues by raising exceptions, which can then be caught and managed gracefully.

## Anticipating Problems: Common File-Related Exceptions

Understanding the specific types of exceptions that can occur during file operations is the first step toward effective error management.

1. FileNotFoundError: This is one of the most frequently encountered file-related exceptions. It is raised when a program attempts to open, read from, or write to a file that simply does not exist at the specified path.

In [12]:
try:
   with open('non_existent_file.txt', 'r') as file:
       content = file.read()
except FileNotFoundError:
   print("Error: The file you tried to open does not exist. Please verify the path.")

Error: The file you tried to open does not exist. Please verify the path.


2. PermissionError: This exception occurs when a program attempts to perform an operation (such as reading, writing, or deleting) on a file or directory for which it lacks the necessary operating system permissions. For instance, trying to write to a protected system file without administrative privileges would typically raise this error.


In [13]:
# This example might require specific file permissions or OS setup to trigger
# On a Unix-like system, try writing to a protected directory like '/root/test.txt' without sudo
try:
   # Attempt to write to a path that typically requires elevated permissions
   with open('C:\Windows\Microsoft\text.txt', 'w') as file:
       file.write("Attempting to write sensitive data.")
except PermissionError:
   print("Error: You do not have sufficient permissions to access or modify this file.")
except Exception as e: # Catch other potential errors if the file doesn't exist etc.
   print(f"An unexpected error occurred: {e}")

An unexpected error occurred: [Errno 22] Invalid argument: 'C:\\Windows\\Microsoft\text.txt'


<>:5: SyntaxWarning: invalid escape sequence '\W'
<>:5: SyntaxWarning: invalid escape sequence '\W'
C:\Users\shant\AppData\Local\Temp\ipykernel_8048\4135058705.py:5: SyntaxWarning: invalid escape sequence '\W'
  with open('C:\Windows\Microsoft\text.txt', 'w') as file:


3. IsADirectoryError: This exception is raised when a program attempts to open a path that points to a directory as if it were a regular file. For example, if my_folder is an existing directory, open('my_folder', 'r') would result in an IsADirectoryError.


In [ ]:
import os
os.makedirs('my_new_directory', exist_ok=True) # Ensure directory exists
try:
   with open('my_new_directory', 'r') as file: # Trying to open a directory as a file
       content = file.read()
except IsADirectoryError:
   print("Error: The path specified is a directory, not a file. Please provide a file path.")
finally:
   # Clean up the created directory for subsequent runs
   if os.path.exists('my_new_directory') and os.path.isdir('my_new_directory'):
       os.rmdir('my_new_directory')

4. IOError / OSError: IOError is a broad category for general input/output errors. In modern Python, specific file-related exceptions like FileNotFoundError, PermissionError, and IsADirectoryError are all subclasses of OSError. Catching OSError can serve as a more general catch-all for various operating system-related problems encountered during file operations.

In [18]:
try:
   # This could potentially raise FileNotFoundError, PermissionError, IsADirectoryError, etc.
   with open('potentially_problematic_file.txt', 'r') as file:
       content = file.read()
except OSError as e: # Catches any OS-related error
   print(f"An operating system error occurred during file access: {e}")

An operating system error occurred during file access: [Errno 2] No such file or directory: 'potentially_problematic_file.txt'


    The relationship between specific file exceptions and the broader OSError base class is an important aspect of Python's exception hierarchy. While FileNotFoundError, PermissionError, and IsADirectoryError offer highly specific details about the nature of a file system problem, allowing for tailored error messages and recovery actions, they all derive from the more general OSError. This hierarchical structure means that catching OSError can simplify except blocks if the goal is to handle any general file system issue without needing to distinguish the exact cause. However, a more refined approach, often considered a best practice, involves catching the most specific exceptions first, followed by more general ones. This strategy enables more precise debugging, allows for distinct responses to different error conditions, and provides clearer feedback to the user or system logs.

## Implementing try-except Blocks for Graceful Handling

The try-except block is Python's fundamental construct for handling exceptions gracefully. It allows a program to anticipate and respond to potential errors without crashing.

    1. The try block encloses the code that might raise an exception.
    2. One or more except blocks follow, each designed to catch and handle a specific type of exception.
    3. An optional else block can be included; its code executes only if no exception occurs within the try block.
    4. An optional finally block, if present, executes unconditionally, regardless of whether an exception occurred or not. While the with statement is the preferred method for ensuring file closure, the finally block remains useful for other cleanup tasks that must always run (e.g., releasing non-file resources, logging completion).

In [19]:
file_name = 'important_data.txt'
# Create a dummy file for demonstration of success path
with open(file_name, 'w') as f:
   f.write("This is important data.")

try:
   with open(file_name, 'r') as f:
       content = f.read()
       print(f"File content: {content}")
except FileNotFoundError:
   print(f"Error: '{file_name}' was not found. Please check the path.")
except PermissionError:
   print(f"Error: You don't have permission to read '{file_name}'.")
except OSError as e: # Catch any other OS-related errors
   print(f"An unexpected OS error occurred with '{file_name}': {e}")
else:
   print(f"Successfully read '{file_name}'. No exceptions were raised.")
finally:
   print("File operation attempt finished. Cleaning up dummy file.")
   if os.path.exists(file_name):
       os.remove(file_name) # Clean up the dummy file

File content: This is important data.
Successfully read 'important_data.txt'. No exceptions were raised.
File operation attempt finished. Cleaning up dummy file.


## Best Practices for Error Handling in File Operations

To build robust and reliable applications that interact with the file system, adhering to certain best practices for error handling is crucial:

   1. Always Use Context Managers (with statement): This is the foremost best practice. Employing with open(...) ensures that files are automatically and properly closed, even if errors occur within the code block. This serves as the primary defense against resource leaks and helps maintain data integrity.
   2. Catch Specific Exceptions First: When structuring try-except blocks, it is advisable to list the most specific exceptions (e.g., FileNotFoundError, PermissionError) before more general ones (e.g., OSError, Exception). This allows for more targeted error messages, enables specific recovery actions, and improves the clarity of your error handling logic.
   3. Log Errors for Debugging: Instead of merely printing error messages to the console, consider implementing a logging mechanism. Logging detailed error information, including the exception type, message, and traceback, is invaluable for debugging and diagnosing issues in production environments.
   4. Proactive File Existence Checks: For read operations, it is often beneficial to proactively check if a file exists using os.path.exists() (covered in Module 8) before attempting to open it. This allows for a more graceful pre-check and can prevent a FileNotFoundError from being raised in the first place. This approach provides an opportunity to inform the user or take alternative action before an exception disrupts the program flow.

The distinction between proactive and reactive error handling is a key strategic consideration in software development. While try-except blocks are essential for reactively catching and handling exceptions after they have occurred, truly robust error management also involves anticipating common problems and preventing them before they even arise. For instance, using os.path.exists() to verify a file's presence before attempting to open it in read mode is a proactive measure. This allows a program to provide more user-friendly feedback, suggest corrective actions, or gracefully adjust its behavior without relying solely on the Python interpreter to raise an exception. Combining reactive exception handling with proactive checks leads to more resilient and user-friendly applications.

# Module 5: Advanced Text File Concepts

Beyond the fundamental operations of reading and writing, a deeper understanding of character encoding and file positioning provides finer-grained control over how text files are handled and interpreted.

## Character Encoding: The encoding Parameter and UTF-8

When text is stored or transmitted, computers do not directly store characters like 'A' or 'π'. Instead, they store numerical representations. Character encoding is the standardized system that maps these numbers to specific characters. If a file is read using an encoding different from the one it was written with, the result can be garbled text (known as "mojibake") or, more commonly, a UnicodeDecodeError.

UTF-8 (Unicode Transformation Format - 8-bit) is the most widely adopted and recommended character encoding standard in modern computing. Its popularity stems from its ability to support virtually all characters and symbols from every language worldwide, making it highly versatile. Furthermore, it is backward-compatible with ASCII, meaning standard English text encoded in ASCII is also valid UTF-8. Python 3 provides native and robust support for UTF-8.

To prevent the common UnicodeDecodeError, it is a strong practice to explicitly specify encoding='utf-8' when opening text files using the open() function. By default, open() might rely on the system's default encoding, which can vary across different operating systems or user configurations, leading to inconsistencies and errors when sharing files or running code in different environments.

In [20]:
# Example: Writing and reading with UTF-8 encoding
text_with_special_chars = "Hello, 世界! こんにちは! 😊 This is a test."
with open('unicode_example.txt', 'w', encoding='utf-8') as f:
   f.write(text_with_special_chars)
print(f"Text written to 'unicode_example.txt' (UTF-8): '{text_with_special_chars}'")

with open('unicode_example.txt', 'r', encoding='utf-8') as f:
   content = f.read()
   print(f"Read with UTF-8 encoding: '{content}'")

# Example of potential UnicodeDecodeError (if file was saved as UTF-8 but read as ASCII)
try:
   with open('unicode_example.txt', 'r', encoding='ascii') as f:
       content_ascii = f.read()
       print(f"Attempted to read with ASCII encoding: '{content_ascii}'")
except UnicodeDecodeError as e:
   print(f"\nCaught UnicodeDecodeError: {e}")
   print("This error occurs when trying to decode characters (like '世界' or '😊') that are not representable in the 'ascii' encoding using the 'ascii' codec.")

# Clean up the dummy file
if os.path.exists('unicode_example.txt'):
   os.remove('unicode_example.txt')

Text written to 'unicode_example.txt' (UTF-8): 'Hello, 世界! こんにちは! 😊 This is a test.'
Read with UTF-8 encoding: 'Hello, 世界! こんにちは! 😊 This is a test.'

Caught UnicodeDecodeError: 'ascii' codec can't decode byte 0xe4 in position 7: ordinal not in range(128)
This error occurs when trying to decode characters (like '世界' or '😊') that are not representable in the 'ascii' encoding using the 'ascii' codec.


The global nature of data and the importance of character encoding are critical considerations in modern software development. In today's interconnected world, applications frequently encounter text that extends beyond the basic English alphabet. Names, addresses, and content can originate from various languages and scripts. Assuming a default encoding, which might be ASCII or a regional standard (e.g., Latin-1), can lead to frustrating UnicodeDecodeError exceptions or, more insidiously, silent data corruption where characters are misinterpreted without an explicit error. Explicitly using UTF-8 is not merely a recommended practice; it is a professional necessity for building applications that are truly global-ready, ensuring that text from diverse linguistic backgrounds is handled consistently and correctly.

## Navigating Files: seek() and tell()

When a file is opened for reading or writing, Python maintains an internal "file pointer" (often conceptualized as a cursor) that indicates the current position within the file where the next read or write operation will occur. The seek() and tell() methods provide direct control and query capabilities over this pointer, enabling non-sequential access to file content.

   ** tell(): This method returns an integer representing the current position of the file pointer within the file. The position is measured in bytes from the beginning of the file.
    seek(offset, whence=0): This method changes the current position of the file pointer.

   ** offset: This is the number of bytes to move the pointer. It can be positive (to move forward) or negative (to move backward).
    whence: This is an optional argument that specifies the reference point for the offset. It can take one of three integer values, often accessed via the os module constants:

           * 0 (or os.SEEK_SET): This is the default. The offset is relative to the beginning of the file.
           * 1 (or os.SEEK_CUR): The offset is relative to the current position of the file pointer.
           * 2 (or os.SEEK_END): The offset is relative to the end of the file. When using this whence, the offset is typically negative to move backward from the end.

It is important to note that when working with text files (not binary files), using seek() with whence values other than os.SEEK_SET (0) can be unreliable due to the complexities of character encoding. For precise byte-level seeking, especially with variable-length encodings like UTF-8, it is generally recommended to open the file in binary mode ('b').

In [22]:
# Create a sample file with known content and length
with open('seek_tell_example.txt', 'w') as f:
   f.write('0123456789abcdefghijklmnopqrstuvwxyz')
print(f"Content of 'seek_tell_example.txt': '0123456789abcdefghijklmnopqrstuvwxyz'")

with open('seek_tell_example.txt', 'r') as f:
   print(f"Initial position: {f.tell()}") # Output: 0

   content1 = f.read(5) # Read first 5 characters
   print(f"Read: '{content1}', Current position: {f.tell()}") # Output: '01234', 5

   f.seek(10) # Move to 10th byte from beginning
   content2 = f.read(5)
   print(f"Seek to 10, Read: '{content2}', Current position: {f.tell()}") # Output: 'abcde', 15

   f.seek(-10, whence = os.SEEK_END) # Move 10 bytes back from the end
   content3 = f.read()
   print(f"Seek -10 from end, Read: '{content3}', Current position: {f.tell()}") # Output: 'wxyz', 36 (end of file)

   f.seek(0) # Go back to the beginning
   print(f"After seeking to beginning, Current position: {f.tell()}") # Output: 0

# Clean up the dummy file
if os.path.exists('seek_tell_example.txt'):
   os.remove('seek_tell_example.txt')

Content of 'seek_tell_example.txt': '0123456789abcdefghijklmnopqrstuvwxyz'
Initial position: 0
Read: '01234', Current position: 5
Seek to 10, Read: 'abcde', Current position: 15


TypeError: TextIOWrapper.seek() takes no keyword arguments

While common reading methods like read() and readlines() abstract away the intricacies of the file pointer, seek() and tell() provide direct, byte-level control. This indicates that these methods are not intended for everyday text file operations but are indispensable for specific, more advanced use cases. For instance, they are crucial when parsing custom binary file formats that have a precise internal structure (e.g., image headers, compressed archives), where jumping to exact byte offsets is necessary. They are also valuable for implementing file indexing, allowing for quick navigation to specific sections of a large file without loading the entire content into memory. Furthermore, in scenarios like resuming interrupted downloads or data transfers, knowing the last written position (tell()) enables a program to seek() to that exact point and continue writing. These functions are powerful tools for optimizing file access and handling complex file structures that extend beyond simple line-by-line or full-content text processing.

# Module 6: Working with Structured Data: CSV Files

CSV (Comma-Separated Values) files represent a ubiquitous format for storing tabular data, making them a common sight in data exchange. Python's built-in csv module significantly simplifies the process of interacting with these files.

## What is a CSV File? (Comma-Separated Values)

A CSV file is essentially a plain-text file that organizes data in a tabular structure, much like a spreadsheet. Each line within the file typically corresponds to a row in a table, and the individual values within that row are separated by a specific character, most commonly a comma. This delimiter gives the format its name: "comma-separated values." Often, the very first line of a CSV file serves as a header row, containing descriptive names for each column, which helps in understanding the data below.

Consider the following example of a people.csv file:
*  name,age,city
*  Alice,30,New York
*  Bob,25,San Francisco
*  Charlie,35,London

In this example, 'name', 'age', and 'city' are the column headers, and each subsequent line represents a record with values separated by commas.

## The csv Module: Your Tool for Tabular Data

Python's standard library includes the csv module, which provides robust and convenient functionality for reading from and writing to CSV files. This module is designed to handle the various complexities inherent in the CSV format, such as values containing the delimiter character (which often require quoting), and different line ending conventions. By abstracting these details, the csv module makes it significantly easier to work with tabular data compared to manually parsing strings.

### Reading CSV Data

The csv module offers two primary ways to read data from CSV files, catering to different preferences for data access.

#### csv.reader: Getting Rows as Lists

The csv.reader object allows for iterating over lines in a CSV file, with each row being returned as a list of strings. This method is straightforward and suitable when the order of columns is known and consistent, or when accessing data by numerical index is preferred.

In [1]:
import csv

# Create a sample CSV file for demonstration
csv_content = """name,age,city
Alice,30,New York
Bob,25,San Francisco"""
with open('people.csv', 'w', newline='') as f: # newline='' is crucial for CSV handling
   f.write(csv_content)

# Reading with csv.reader
print("--- Reading 'people.csv' with csv.reader ---")
with open('people.csv', 'r', newline='') as csv_file:
   csv_reader = csv.reader(csv_file, delimiter=',')
   line_count = 0
   for row in csv_reader:
       if line_count == 0:
           print(f'Column names: {", ".join(row)}')
       else:
           print(f'\t{row} is {row} years old and lives in {row}.')
       line_count += 1

--- Reading 'people.csv' with csv.reader ---
Column names: name, age, city
	['Alice', '30', 'New York'] is ['Alice', '30', 'New York'] years old and lives in ['Alice', '30', 'New York'].
	['Bob', '25', 'San Francisco'] is ['Bob', '25', 'San Francisco'] years old and lives in ['Bob', '25', 'San Francisco'].


# DISCONTINUED (CSV AND JSON FILE HANDLING IN SEPERATE)